# Übung 3

## 1) Gesichtserkennung mit Naive Bayes

In [15]:
import pandas as pd
import numpy as np
import matplotlib as mp
import tarfile
import os.path
import urllib.request
import skimage
import scipy as sp
from random import shuffle

In [5]:
filename = "lfw-funneled.tgz"
url = "http://vis-www.cs.umass.edu/lfw/" + filename
if not os.path.isfile("../02/" + filename):
    urllib.request.urlretrieve(url, filename)
    dateDownloaded = !date
    print(dateDownloaded)

In [6]:
tar = tarfile.open("../02/" + filename)
tar.extractall()
tar.close()
foldername = "lfw_funneled"

In [71]:
persons = {}
for personName in os.listdir(foldername):
    current_person = foldername + "/" + personName
    if os.path.isdir(current_person):
        if len(os.listdir(current_person)) >= 70:
            shuffled_dir_list = os.listdir(current_person)
            shuffle(shuffled_dir_list)
            persons.update({personName: [current_person +"/"+item for item in shuffled_dir_list] })
persons.keys()

dict_keys(['Ariel_Sharon', 'Colin_Powell', 'Gerhard_Schroeder', 'George_W_Bush', 'Tony_Blair', 'Hugo_Chavez', 'Donald_Rumsfeld'])

In [72]:
from skimage import io
from skimage import transform

resized_images = []
resized_names = []
resized_last_images = []
resized_last_names = []

for person_name, person in zip(persons.keys(), persons.values()):
    path = person.copy()
    size = (int)(len(path) * 0.6)
    for picture_path in path:
        picture = io.imread(picture_path,  as_grey=True)
        picture = picture[55:195, 75:175]
        resized_image = transform.resize(picture, ((int)(picture.shape[0]/8), (int)(picture.shape[1]/8))).flatten()
        if picture_path not in path[size:len(path)]:
            resized_images.append(resized_image)
            resized_names.append(person_name)
        else:
            resized_last_images.append(resized_image)
            resized_last_names.append(person_name)

resized_images = np.array(resized_images)
resized_names = np.array(resized_names)
resized_last_images = np.array(resized_last_images)
resized_last_names = np.array(resized_last_names)
        
resized_images.shape, resized_names.shape, resized_last_images.shape, resized_last_names.shape

((770, 204), (770,), (518, 204), (518,))

Führen Sie anschließend eine Hauptkomponentenanalyse auf den Trainingsdaten durch und projizieren Sie sowohl Trainings- als auch Testbilder auf die ersten 7 Eigengesichter.

In [73]:
training_data_mean = resized_images.mean(axis=0)
centered_training_data = resized_images.copy()
centered_training_data=centered_training_data-training_data_mean

U, D, Vt = np.linalg.svd(resized_images)

eigenvalues = np.square(D)

centered_test_data = resized_last_images.copy()
centered_test_data=centered_test_data-training_data_mean

test_images = np.dot(Vt[:7], centered_test_data.T).T
training_images = np.dot(Vt[:7], centered_training_data.T).T

test_images.shape, training_images.shape

((518, 7), (770, 7))

Trainieren Sie Ihren GNB-Klassifikator auf dem Trainingsdatensatz als “George-W.-Bush-Detektor”, d.h. alle zu dieser Person gehörigen Bilder werden mit 1 gelabelt, alle sonstigen mit –1

In [74]:
training_names = resized_names
training_labels = []
for training_image, training_name in zip(training_images, training_names):
    if training_name == "George_W_Bush":
        training_labels.append(1)
    else:
         training_labels.append(-1) 
            
test_names = resized_last_names  
test_labels = []
for test_image, test_name in zip(test_images, test_names):
    if test_name == "George_W_Bush":
        test_labels.append(1)
    else:
         test_labels.append(-1) 

training_labels.count(1), training_labels.count(-1), test_labels.count(1), test_labels.count(-1)

(318, 452, 212, 306)

In [75]:
training_not_bush = np.empty((0,training_images.shape[1]))
training_bush =  np.empty((0,training_images.shape[1]))
for training_name, training_label, training_image in zip(training_names, training_labels, training_images):
    if training_label == 1:
        training_bush = np.vstack((training_bush, training_image))
    else:
        training_not_bush = np.vstack((training_not_bush, training_image))  

Folie 18: Mittelwert und Standardabweichung

In [76]:
training_not_bush_mean = np.mean(training_not_bush, axis=0)
training_bush_mean =np.mean(training_bush, axis=0)
test_images_mean = np.mean(test_images, axis=0)

training_not_bush_std = np.std(training_not_bush, axis=0)
training_bush_std =np.std(training_bush, axis=0)
test_images_std = np.std(test_images, axis=0)

training_not_bush_mean, training_bush_mean, test_images_mean, training_not_bush_std, training_bush_std, test_images_std

(array([-0.00976962,  0.01566169,  0.0215188 ,  0.01556443, -0.02077515,
         0.09624833, -0.04107474]),
 array([ 0.01388638, -0.02226127, -0.03058647, -0.02212303,  0.02952946,
        -0.1368058 ,  0.05838296]),
 array([ 0.06326181,  0.00674253, -0.06381036,  0.03731975, -0.0109283 ,
        -0.00378037,  0.00508846]),
 array([ 0.91536605,  0.75333914,  0.77143178,  0.6024515 ,  0.4806122 ,
         0.41443685,  0.35729905]),
 array([ 0.91886927,  0.82191197,  0.5667829 ,  0.60497727,  0.48746507,
         0.32158683,  0.37280695]),
 array([ 0.96399672,  0.83565838,  0.66363332,  0.55807439,  0.47007326,
         0.40128874,  0.34413868]))

A-Priori Wahrscheinlichkeiten

In [77]:
apriori_bush = training_bush.shape[0] / (training_bush.shape[0]+training_not_bush.shape[0])
apriori_not_bush = training_not_bush.shape[0] / (training_bush.shape[0]+training_not_bush.shape[0])
apriori_bush, apriori_not_bush

(0.412987012987013, 0.587012987012987)

In [78]:
gaussian_distribution_bush = sp.stats.norm(training_bush_mean, training_bush_std).pdf(test_images)
gaussian_distribution_not_bush = sp.stats.norm(training_not_bush_mean, training_not_bush_std).pdf(test_images)

Wahrscheinlichkeit, dass Testbild Bush ist

In [79]:
probability_bush = np.prod(gaussian_distribution_bush, axis=1) * apriori_bush
probability_not_bush = np.prod(gaussian_distribution_not_bush, axis=1) * apriori_not_bush
probability_bush.shape, probability_not_bush.shape

((518,), (518,))

Werten Sie Ihren Klassifikator sowohl auf den Trainings- wie auf den unabhängigen Testdaten aus

Bestimmen Sie dafür jeweils die Detektionswahrscheinlichkeit, Richtig-Negativ-Rate, Fehlalarmrate und Falsch-Negativ-Rate

In [80]:
false_positive = 0
false_negative = 0
true_positive = 0
true_negative = 0

i = 0

classifications = (probability_bush / probability_not_bush)-1
for result in classifications:
    if result < 0:
        if test_labels[i] == -1:
            true_negative +=1
        else:
            false_negative +=1
    elif result > 0:
        if test_labels[i] == 1:
            true_positive +=1
        else:
            false_positive +=1
    elif result == 0:
        pass
    i+=1

print("Wahrscheinlichkeit für richtige Klassifikation: ", (true_positive + true_negative)/probability_bush.shape[0]*100)
true_positive, true_negative, false_positive, false_negative

Wahrscheinlichkeit für richtige Klassifikation:  63.51351351351351


(124, 205, 101, 88)

## Konfiguration

In [81]:
%load_ext version_information
%version_information numpy, pandas, matplotlib, skimage

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 3.5.2 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 5.1.0
OS Linux 4.2.0 27 generic x86_64 with debian jessie sid
numpy 1.11.1
pandas 0.18.1
matplotlib 1.5.3
skimage 0.12.3
Sun Nov 27 15:17:03 2016 CET